# Segmenting and Clustering Neighborhoods in Toronto
Notebook for the peer-graded assignment of Week 3 of the Applied Data Science Capstone. In this assignment, the neighborhoods in the city of Toronto are explored, segmented and divided into clusters. 

### Initializing the required libraries 

In [6]:
!pip install beautifulsoup4
!pip install lxml 

import requests #library to handle requests 
import pandas as pd #library for data analysis 
import numpy as np #library to handle data as vectors 
import random #library for pseudo-random number generation 

# module to convert an address into latitude and longitude values
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
print ('Nominatim installed')

#Libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML 
from IPython.display import display_html 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported')

Nominatim installed
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\nmilella\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       he774522_0         5.7 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         6.4 MB

The following NEW packages will be INSTALLED:

  altair 

###  Scraping from Wikipedia 

In [7]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
print(soup.title)
tab = str(soup.table)
display_html(tab, raw= True)

<title>List of postal codes of Canada: M - Wikipedia</title>


Postal Code,Borough,Neighbourhood
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
M8A,Not assigned,Not assigned
M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
M1B,Scarborough,"Malvern, Rouge"


### Pandas dataframe and data cleaning

In [15]:
#Crearing pandas df
dfs = pd.read_html(tab)
df = dfs[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [22]:
# Dropping raws where Borough is 'Not assigned'
df1 = df[df.Borough != 'Not assigned']

# Grouping Neighbourhood with same Postal Code
df2 = df1.groupby(['Postal Code', 'Borough'], sort = False).agg(','.join)
df2.reset_index(inplace = True)

# Replacing Neighbourhood 'Not assigned' with Borough
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned', df2['Borough'], df2['Neighbourhood'])

df2.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [24]:
df2.shape

(103, 3)

In [33]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

URLError: <urlopen error [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions>